In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

def detect_and_compute(img, algorithm):
    if algorithm == 'SIFT':
        detector = cv2.SIFT_create()
    elif algorithm == 'ORB':
        detector = cv2.ORB_create()
    elif algorithm == 'AKAZE':
        detector = cv2.AKAZE_create()
    elif algorithm == 'BRISK':
        detector = cv2.BRISK_create()
    else:
        raise ValueError(f"Unsupported algorithm: {algorithm}")
    
    keypoints, descriptors = detector.detectAndCompute(img, None)
    return keypoints, descriptors

def match_features(des1, des2, algorithm):
    if algorithm in ['ORB', 'BRISK']:
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
    else:
        bf = cv2.BFMatcher()
    
    if des1 is not None and des2 is not None and len(des1) > 0 and len(des2) > 0:
        matches = bf.knnMatch(des1, des2, k=2)
        good_matches = []
        for m_n in matches:
            if len(m_n) == 2:
                m, n = m_n
                if m.distance < 0.75 * n.distance:
                    good_matches.append(m)
        return good_matches
    else:
        return []

def evaluate_image_pair(img1, img2, algorithm):
    kp1, des1 = detect_and_compute(img1, algorithm)
    kp2, des2 = detect_and_compute(img2, algorithm)
    
    matches = match_features(des1, des2, algorithm)
    
    return len(kp1), len(kp2), len(matches)

def process_image_pairs(original_dir, damaged_dir, algorithms):
    results = {alg: {'keypoints1': [], 'keypoints2': [], 'matches': []} for alg in algorithms}
    
    original_images = sorted(os.listdir(original_dir))
    damaged_images = sorted(os.listdir(damaged_dir))
    
    for orig_img, dam_img in tqdm(zip(original_images, damaged_images), total=len(original_images)):
        img1 = cv2.imread(os.path.join(original_dir, orig_img), cv2.IMREAD_GRAYSCALE)
        img2 = cv2.imread(os.path.join(damaged_dir, dam_img), cv2.IMREAD_GRAYSCALE)
        
        if img1 is None or img2 is None:
            continue
        
        for alg in algorithms:
            kp1, kp2, matches = evaluate_image_pair(img1, img2, alg)
            results[alg]['keypoints1'].append(kp1)
            results[alg]['keypoints2'].append(kp2)
            results[alg]['matches'].append(matches)
    
    return results

def calculate_averages(results):
    avg_results = {}
    for alg, data in results.items():
        avg_results[alg] = {
            'avg_keypoints1': np.mean(data['keypoints1']),
            'avg_keypoints2': np.mean(data['keypoints2']),
            'avg_matches': np.mean(data['matches']),
            'match_ratio': np.mean(data['matches']) / np.mean(data['keypoints1']) if np.mean(data['keypoints1']) > 0 else 0
        }
    return avg_results

def plot_results(avg_results):
    algorithms = list(avg_results.keys())
    metrics = ['avg_keypoints1', 'avg_keypoints2', 'avg_matches', 'match_ratio']
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))
    
    x = np.arange(len(algorithms))
    width = 0.2
    
    for i, metric in enumerate(['avg_keypoints1', 'avg_keypoints2', 'avg_matches']):
        values = [avg_results[alg][metric] for alg in algorithms]
        ax1.bar(x + i*width, values, width, label=metric)
    
    ax1.set_ylabel('Average Count')
    ax1.set_title('Keypoints and Matches Comparison')
    ax1.set_xticks(x + width)
    ax1.set_xticklabels(algorithms, rotation=45)
    ax1.legend()
    
    match_ratios = [avg_results[alg]['match_ratio'] for alg in algorithms]
    ax2.bar(algorithms, match_ratios)
    ax2.set_ylabel('Match Ratio')
    ax2.set_title('Match Ratio Comparison')
    ax2.set_ylim(0, 1)
    
    plt.tight_layout()
    plt.savefig('./algorithm_performance_comparison.png')
    plt.close()

# Main execution
original_dir = "./data/original/"
damaged_dir = "./data/damaged/"
algorithms = ['SIFT', 'ORB', 'AKAZE', 'BRISK']

results = process_image_pairs(original_dir, damaged_dir, algorithms)
avg_results = calculate_averages(results)

plot_results(avg_results)

print("Average Results:")
for alg, res in avg_results.items():
    print(f"{alg}:")
    print(f"  Average Keypoints in original images: {res['avg_keypoints1']:.2f}")
    print(f"  Average Keypoints in damaged images: {res['avg_keypoints2']:.2f}")
    print(f"  Average Matches: {res['avg_matches']:.2f}")
    print(f"  Match Ratio: {res['match_ratio']:.4f}")
    print()

print("Performance comparison graph saved as 'algorithm_performance_comparison.png'")

100%|██████████| 1763/1763 [04:55<00:00,  5.96it/s]


Average Results:
SIFT:
  Average Keypoints in original images: 402.48
  Average Keypoints in damaged images: 362.93
  Average Matches: 174.23
  Match Ratio: 0.4329

ORB:
  Average Keypoints in original images: 407.15
  Average Keypoints in damaged images: 418.77
  Average Matches: 151.84
  Match Ratio: 0.3729

AKAZE:
  Average Keypoints in original images: 158.08
  Average Keypoints in damaged images: 153.01
  Average Matches: 57.86
  Match Ratio: 0.3660

BRISK:
  Average Keypoints in original images: 1090.72
  Average Keypoints in damaged images: 1895.89
  Average Matches: 425.57
  Match Ratio: 0.3902

Performance comparison graph saved as 'algorithm_performance_comparison.png'


In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

def detect_and_compute(img, algorithm):
    if algorithm == 'SIFT':
        detector = cv2.SIFT_create()
    elif algorithm == 'ORB':
        detector = cv2.ORB_create()
    elif algorithm == 'AKAZE':
        detector = cv2.AKAZE_create()
    elif algorithm == 'BRISK':
        detector = cv2.BRISK_create()
    else:
        raise ValueError(f"Unsupported algorithm: {algorithm}")
    
    keypoints, descriptors = detector.detectAndCompute(img, None)
    return keypoints, descriptors

def match_features(des1, des2, algorithm):
    if algorithm in ['ORB', 'BRISK']:
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
    else:
        bf = cv2.BFMatcher()
    
    if des1 is not None and des2 is not None and len(des1) > 0 and len(des2) > 0:
        matches = bf.knnMatch(des1, des2, k=2)
        good_matches = []
        for m_n in matches:
            if len(m_n) == 2:
                m, n = m_n
                if m.distance < 0.75 * n.distance:
                    good_matches.append(m)
        return good_matches
    else:
        return []

def evaluate_image_pair(img1, img2, algorithm):
    kp1, des1 = detect_and_compute(img1, algorithm)
    kp2, des2 = detect_and_compute(img2, algorithm)
    
    matches = match_features(des1, des2, algorithm)
    
    return len(kp1), len(kp2), len(matches)

def process_image_pairs(original_dir, damaged_dir, algorithms):
    results = {alg: {'keypoints1': [], 'keypoints2': [], 'matches': []} for alg in algorithms}
    
    original_images = sorted(os.listdir(original_dir))
    damaged_images = sorted(os.listdir(damaged_dir))
    
    for orig_img, dam_img in tqdm(zip(original_images, damaged_images), total=len(original_images)):
        img1 = cv2.imread(os.path.join(original_dir, orig_img), cv2.IMREAD_GRAYSCALE)
        img2 = cv2.imread(os.path.join(damaged_dir, dam_img), cv2.IMREAD_GRAYSCALE)
        
        if img1 is None or img2 is None:
            continue
        
        for alg in algorithms:
            kp1, kp2, matches = evaluate_image_pair(img1, img2, alg)
            results[alg]['keypoints1'].append(kp1)
            results[alg]['keypoints2'].append(kp2)
            results[alg]['matches'].append(matches)
    
    return results

def calculate_averages(results):
    avg_results = {}
    for alg, data in results.items():
        avg_results[alg] = {
            'avg_keypoints1': np.mean(data['keypoints1']),
            'avg_keypoints2': np.mean(data['keypoints2']),
            'avg_matches': np.mean(data['matches']),
            'match_ratio': np.mean(data['matches']) / np.mean(data['keypoints1']) if np.mean(data['keypoints1']) > 0 else 0
        }
    return avg_results

def plot_results(avg_results):
    algorithms = list(avg_results.keys())
    metrics = ['avg_keypoints1', 'avg_keypoints2', 'avg_matches', 'match_ratio']
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))
    
    x = np.arange(len(algorithms))
    width = 0.2
    
    for i, metric in enumerate(['avg_keypoints1', 'avg_keypoints2', 'avg_matches']):
        values = [avg_results[alg][metric] for alg in algorithms]
        ax1.bar(x + i*width, values, width, label=metric)
    
    ax1.set_ylabel('Average Count')
    ax1.set_title('Keypoints and Matches Comparison')
    ax1.set_xticks(x + width)
    ax1.set_xticklabels(algorithms, rotation=45)
    ax1.legend()
    
    match_ratios = [avg_results[alg]['match_ratio'] for alg in algorithms]
    ax2.bar(algorithms, match_ratios)
    ax2.set_ylabel('Match Ratio')
    ax2.set_title('Match Ratio Comparison')
    ax2.set_ylim(0, 1)
    
    plt.tight_layout()
    plt.savefig('./algorithm_performance_comparison.png')
    plt.close()

# Main execution
original_dir = "./data/original/"
damaged_dir = "./data/damaged/"
algorithms = ['SIFT', 'ORB', 'AKAZE', 'BRISK']

results = process_image_pairs(original_dir, damaged_dir, algorithms)
avg_results = calculate_averages(results)

plot_results(avg_results)

print("Average Results:")
for alg, res in avg_results.items():
    print(f"{alg}:")
    print(f"  Average Keypoints in original images: {res['avg_keypoints1']:.2f}")
    print(f"  Average Keypoints in damaged images: {res['avg_keypoints2']:.2f}")
    print(f"  Average Matches: {res['avg_matches']:.2f}")
    print(f"  Match Ratio: {res['match_ratio']:.4f}")
    print()

print("Performance comparison graph saved as 'algorithm_performance_comparison.png'")

100%|██████████| 1763/1763 [03:02<00:00,  9.63it/s]


Average Results:
SIFT:
  Average Keypoints in original images: 402.48
  Average Keypoints in damaged images: 362.93
  Average Matches: 174.23
  Match Ratio: 0.4329

ORB:
  Average Keypoints in original images: 407.15
  Average Keypoints in damaged images: 418.77
  Average Matches: 151.84
  Match Ratio: 0.3729

AKAZE:
  Average Keypoints in original images: 158.08
  Average Keypoints in damaged images: 153.01
  Average Matches: 57.86
  Match Ratio: 0.3660

BRISK:
  Average Keypoints in original images: 1090.72
  Average Keypoints in damaged images: 1895.89
  Average Matches: 425.57
  Match Ratio: 0.3902

Performance comparison graph saved as 'algorithm_performance_comparison.png'


In [ ]:
import React from 'react';
import { RadarChart, PolarGrid, PolarAngleAxis, PolarRadiusAxis, Radar, Legend } from 'recharts';

const data = [
  { characteristic: '속도', SIFT: 2, ORB: 5, AKAZE: 4, BRISK: 5 },
  { characteristic: '정확도', SIFT: 5, ORB: 3, AKAZE: 4, BRISK: 3 },
  { characteristic: '스케일 불변성', SIFT: 5, ORB: 3, AKAZE: 4, BRISK: 3 },
  { characteristic: '회전 불변성', SIFT: 5, ORB: 4, AKAZE: 4, BRISK: 4 },
  { characteristic: '메모리 효율성', SIFT: 2, ORB: 5, AKAZE: 4, BRISK: 4 },
  { characteristic: '특징점 수', SIFT: 4, ORB: 3, AKAZE: 3, BRISK: 5 },
];

const AlgorithmComparisonChart = () => (
  <div className="flex flex-col items-center w-full max-w-3xl mx-auto">
    <h2 className="text-2xl font-bold mb-4">특징점 검출 알고리즘 성능 비교</h2>
    <RadarChart outerRadius={150} width={600} height={400} data={data}>
      <PolarGrid />
      <PolarAngleAxis dataKey="characteristic" />
      <PolarRadiusAxis angle={30} domain={[0, 5]} />
      <Radar name="SIFT" dataKey="SIFT" stroke="#8884d8" fill="#8884d8" fillOpacity={0.6} />
      <Radar name="ORB" dataKey="ORB" stroke="#82ca9d" fill="#82ca9d" fillOpacity={0.6} />
      <Radar name="AKAZE" dataKey="AKAZE" stroke="#ffc658" fill="#ffc658" fillOpacity={0.6} />
      <Radar name="BRISK" dataKey="BRISK" stroke="#ff8042" fill="#ff8042" fillOpacity={0.6} />
      <Legend />
    </RadarChart>
    <p className="mt-4 text-sm text-gray-600">
      주의: 이 그래프는 일반적인 성능 특성을 나타내며, 실제 성능은 사용 사례와 구현에 따라 다를 수 있습니다.
    </p>
  </div>
);

export default AlgorithmComparisonChart;